# Topic Analysis - Project Assignment Group 3 

In [42]:
from datasets import load_dataset, get_dataset_config_names
import pyLDAvis
import pandas as pd 
import spacy
from tqdm.auto import tqdm
import random
from sklearn.model_selection import train_test_split
import pycountry
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import pickle 
import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS 
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models as gensimvis

from sklearn.metrics import classification_report, confusion_matrix

### Dataset Inspection and Content Extraction 


In the first part of our Topic Analysis section of the project, we will extract all the sections in the database and count them to see what each news article is assigned to. In this way, we can choose our topics and see what we are first counting.

We will notice that the _section_ column in the dataset is not helpful, since it groups the articles in a sparse way and does not provide useful labels for our topic classification.

Thus, we will have to preprocess each article by first extracting the _content_ column from each (ignoring all the others) In this way we will feed our LDA model for topic analysis. 

In [ ]:
subfolders = get_dataset_config_names("RealTimeData/bbc_news_alltime")

all_sections = []

for month in subfolders:
    dataset = load_dataset("RealTimeData/bbc_news_alltime", month, split="train")
    if "section" in dataset.column_names:
        all_sections.extend(dataset["section"])

section_df = pd.DataFrame(all_sections, columns=["section"])
section_counts = section_df["section"].value_counts(dropna=False).reset_index()
section_counts.columns = ["Topic", "Count"]


In [ ]:
section_counts = section_df['section'].value_counts(dropna=False).reset_index()
section_counts.columns = ['Topic', 'Count']


pd.set_option('display.max_rows', None)
print(section_counts)

Loading SpaCy, getting the dataset and splitting between test and train folders (70/30)

In [ ]:
subfolders = get_dataset_config_names("RealTimeData/bbc_news_alltime")
preprocessing = spacy.load("en_core_web_sm", disable=["ner", "parser"])
preprocessing.add_pipe("sentencizer")
random.seed(42)
selected_months = random.sample(subfolders, k=20)

#Splitting the folders in train and test sets based on months folders 
train_months, test_months = train_test_split(selected_months, test_size=0.3, random_state=42)

In [ ]:
train_sentences = []

# Looping over the months folders
for month in tqdm(train_months, desc="Month"): 
    dataset = load_dataset("RealTimeData/bbc_news_alltime", month, split="train")
    # Looping over the various articles
    content = [article["content"] for article in dataset] 
    # Getting the Doc object of the article and looping over them splittin the sentences
    for doc in preprocessing.pipe(content, batch_size=32):
        for sent in doc.sents:
            text = sent.text.strip()
            if text:
                print(text)
                train_sentences.append(text)

print(train_sentences[:5])

In [6]:
for x in train_sentences[:5]:
    print(x)

Theresa May was in Poland to sign a defence treaty with the country

Theresa May has sought to reassure Polish people living in the UK that they are still welcome after Brexit.
Speaking on a trip to Warsaw to sign a new defence treaty with the country, the PM said the one million Polish residents were a "strong part of [UK] society".
She promised a "simple" and "easy" process to get "settled status" to remain after the UK leaves the EU.
The trip comes after Mrs May sacked one of her closest allies, Damian Green.
She asked him to leave after he made "misleading" statements about claims pornography was found on his parliamentary computer.


In [ ]:
# Now we are applying the same procedure for the test set in the test_months folder 
test_sentences = []

for month in tqdm(test_months, desc="Months Analyzed"): 
    dataset = load_dataset("RealTimeData/bbc_news_alltime", month, split="train")
    # Looping over the various articles
    content = [article["content"] for article in dataset] 
    # Getting the Doc object of the article and looping over them splittin the sentences
    for doc in preprocessing.pipe(content, batch_size=32):
        for sent in doc.sents:
            text = sent.text.strip()
            if text:
                test_sentences.append(text)

Months Analyzed:   0%|          | 0/6 [00:00<?, ?it/s]

In [11]:
for x in test_sentences[:5]:
    print(x)

Amiram Cooper, Guy Gilboa-Dalal and Tsachi Idan are being held in Gaza

Israel says 129 people remain unaccounted for after they were abducted and taken to Gaza during the October 7 attacks by Hamas.
Of these, Israel says that 22 are believed to be dead.
A group representing the families of hostages says that Gadi Haggai, 73, is now believed to have died in Gaza.
An estimated 240 people were taken prisoner, but 105 were later released by Hamas during a six-day ceasefire at the end of November.
These are the stories of those hostages who are still being held, which have either been confirmed by the BBC or credibly reported.


### Data Preprocessing using SpaCy

In this **preprocessing** stage, we transform raw sentences into clean, *lemmatized* tokens optimized for topic modeling. Specifically, we use **SpaCy** to tokenize and lemmatize each sentence, removing stopwords, punctuation, short words, and numbers. 

We also exclude tokens based on their linguistic roles (pronouns, determiners, prepositions, auxiliaries) and named entities (people, organizations, locations, dates, times). Then, we filter out domain-specific noise such as media sources ("BBC", "Reuters"), common journalistic fillers ("said", "today"), possessive pronouns, and country names using a custom exclusion list. 

The resulting cleaned tokens are then structured into a **Gensim-compatible dictionary and corpus**, ready for training the LDA model. 
This also takes into account the labels in the test set provided for the project. In fact, our goal is to align as closely as possible to the topics shown in that test set. Thus, we cut out all the noise we think we may encounter.

In [23]:
token_pos_ner = spacy.load("en_core_web_sm", disable=["parser"])
stopwords = token_pos_ner.Defaults.stop_words

countries_list= {country.name.lower() for country in pycountry.countries}

# Excluding some words that are indeed part of newspapers articles but not necessary
custom_stop = {
    "bbc","reuters","cnn","guardian","nytimes","telegraph","aljazeera",
    "news","press","article","media","coverage","broadcast","report","headline",
    "says","said","told","claim","claimed","statement",
    "thing","stuff","someone","anyone","everyone","something","everything",
    "kind","sort","part","place","area","around"
}

# Unifying the various exclusions we listed above (costumized)
all_exclusions = stopwords.union(custom_stop).union(countries_list)
pos_exclusions  = {"PRON","DET","ADP","AUX","INTJ"}
entities_excl   = {"PERSON","ORG","GPE","LOC","DATE","TIME"}


def preprocessing(sentence):
    """
    This function preprocesses sentences coming from test/train lists. 
    It removes the PoS tags adn entity labels listed above. It also removes unecessary words 
    that are not helpful for our topic analysis
    """

    token_lists, cleaned_strings = [], []

    for i in tqdm(range(0, len(train_sentences), 500), desc="Sentence batches"):

        batch_sents = train_sentences[i : i + 500]

        for article in token_pos_ner.pipe(batch_sents, batch_size=16): 
            tokens = [token.lemma_.lower() for token in article if token.lemma_.lower() not in all_exclusions
                and len(token) > 3
                and not token.is_punct
                and not token.like_num
                and token.pos_ not in pos_exclusions
                and token.ent_type_ not in entities_excl]
            token_lists.append(tokens)
            cleaned_strings.append(" ".join(tokens))

    return token_lists, cleaned_strings

def preprocessing_test(sentence):
    """
    This function preprocesses sentences coming from test/train lists. 
    It removes the PoS tags adn entity labels listed above. It also removes unecessary words 
    that are not helpful for our topic analysis
    """

    token_lists, cleaned_strings = [], []

    for i in tqdm(range(0, len(test_sentences), 500), desc="Sentence batches"):

        batch_sents = test_sentences[i : i + 500]

        for article in token_pos_ner.pipe(batch_sents, batch_size=16): 
            tokens = [token.lemma_.lower() for token in article if token.lemma_.lower() not in all_exclusions
                and len(token) > 3
                and not token.is_punct
                and not token.like_num
                and token.pos_ not in pos_exclusions
                and token.ent_type_ not in entities_excl]
            token_lists.append(tokens)
            cleaned_strings.append(" ".join(tokens))

    return token_lists, cleaned_strings

In [ ]:
train_tokens, train_tokens_string = preprocessing(train_sentences)


In [24]:
test_tokens, test_tokens_string = preprocessing_test(test_sentences)

Sentence batches:   0%|          | 0/443 [00:00<?, ?it/s]

### Checkpoint 

(The Variable is too heavy to reload the process every time, so we saved the variables in a pkl file we can use it again)

In [ ]:
# Saving the variables for later usage, in order to not loose the preprocessing 
with open("train_processed.pkl", "wb") as f: 
    pickle.dump({"tokens": train_tokens, 
                 "strings": train_tokens_string}, f) 
    
# Saving the variables for later usage, in order to not loose the preprocessing 
with open("test_processed.pkl", "wb") as f: 
    pickle.dump({"tokens": test_tokens, 
                 "strings": test_tokens_string}, f)

In [ ]:
with open("train_processed.pkl", "rb") as f:
    data = pickle.load(f)

train_tokens = data["tokens"]
train_token_strings = data["strings"]

with open("test_processed.pkl", "rb") as f:
    data = pickle.load(f)

train_tokens = data["tokens"]
train_token_strings = data["strings"]

In [19]:
for sentence in (train_tokens[:5]): 
    print(sentence)

['theresa', 'sign', 'defence', 'treaty', 'country', 'theresa', 'seek', 'reassure', 'polish', 'people', 'live', 'welcome']
['speak', 'trip', 'sign', 'defence', 'treaty', 'country', 'polish', 'resident', 'strong', 'society']
['promise', 'simple', 'easy', 'process', 'settle', 'status', 'remain', 'leave']
['trip', 'come', 'sack', 'close', 'ally']
['ask', 'leave', 'misleading', 'pornography', 'find', 'parliamentary', 'computer']


### Training LDA model

In [ ]:
dictionary = corpora.Dictionary(train_tokens)
dictionary.filter_extremes(no_below=10, no_above=0.5)
bow_dict = [dictionary.doc2bow(doc) for doc in train_tokens]

lda_model = models.LdaModel(corpus = bow_dict, id2word = dictionary, num_topics = 10, passes = 10, random_state = 42, eval_every = 1, per_word_topics = True)


In [ ]:
lda_model.save("lda_bbc_news.model")

In [ ]:
lda_model = models.LdaModel.load("lda_bbc_news.model")

In [44]:
def show_top_words(model, dictionary, n_words=10):
    print("TOP WORDS PER TOPIC\n" + "-"*30)
    for tid, topic in model.show_topics(num_topics=model.num_topics,
                                        num_words=n_words,
                                        formatted=False):
        words = ", ".join([w for w, p in topic])
        print(f"Topic {tid:>2}  {words}")

show_top_words(lda_model, dictionary, n_words=30)

pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, bow_dict, dictionary)
vis

TOP WORDS PER TOPIC
------------------------------
Topic  0  content, site, responsible, external, continue, view, miss, choose, accept, original, baby, conservatives, learn, financial, period, couple, daughter, girl, available, bear, allow, visit, train, station, west, simply, trust, datum, absolutely, athlete
Topic  1  provide, follow, post, title, story, contain, send, mother, content, twitter, facebook, skip, hand, champion, stop, penalty, receive, return, rape, write, idea, incident, pick, prison, reason, supporter, develop, foreign, success, woman
Topic  2  party, government, minister, vote, election, deal, leader, prime, secretary, plan, decision, company, business, political, question, firm, future, rise, support, rule, conservative, tell, announce, stand, cost, market, money, rate, country, price
Topic  3  think, look, add, people, time, feel, medium, tell, forward, work, social, long, good, speak, come, ask, matter, expect, know, information, message, public, process, general

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.138478  0.008984       1        1  14.043573
5     -0.160570  0.006284       2        1  12.458512
7     -0.084029 -0.118386       3        1  11.757814
3     -0.178690  0.042465       4        1  11.747048
8     -0.091099  0.154486       5        1  10.581952
6     -0.122314  0.125100       6        1  10.494703
9     -0.022225 -0.184324       7        1  10.489676
1      0.191601 -0.323006       8        1   6.934542
0      0.304814  0.066805       9        1   5.901774
4      0.300989  0.221592      10        1   5.590407, topic_info=          Term          Freq         Total Category  logprob  loglift
175       play  24779.000000  24779.000000  Default  30.0000  30.0000
320       need  20085.000000  20085.000000  Default  29.0000  29.0000
837    content  22767.000000  22767.000000  Default  28.0000  28.0000
53        want  17915.000000  17915.000000  Default  27.0000  27.0000
330      video  17867.000000  17867.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
1503    severe    988.968655    989.923074  Topic10  -5.5496   2.8832
2979    stress    984.448031    985.402414  Topic10  -5.5542   2.8831
838   external   6653.201686  17728.595648  Topic10  -3.6434   1.9040
951     accept   3593.468874   8918.443878  Topic10  -4.2594   1.9751
1440     image   1187.636232   2359.493995  Topic10  -5.3666   2.1976

[454 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
2484      7  0.999739       .from
941       9  0.999200  absolutely
331       3  0.999761       abuse
951       9  0.596965      accept
951      10  0.402873      accept
...     ...       ...         ...
682       4  0.999532        year
638       2  0.595173       young
638       3  0.259804       young
638       4  0.144952       young
4804     10  0.999227       youth

[544 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 6, 8, 4, 9, 7, 10, 2, 1, 5])